# Transient Single-Group Pedestrian Flow - Mathematical Formulation

## 1. Introduction

This notebook implements a transient (time-dependent) single-group pedestrian flow model based on the Hughes model with regularization. The model couples a time-dependent continuity equation for pedestrian density with a Helmholtz equation for path planning.

---

## 2. Strong Form Formulation

### 2.1 Domain and Variables

- **Domain:** $\Omega \subset \mathbb{R}^2$ (the region where pedestrians move)
- **Boundary:** $\partial\Omega = \Gamma_{\text{walls}} \cup \Gamma_{\text{exits}} \cup \Gamma_{\text{entries}}$
- **Time interval:** $t \in [0, T]$

**Primary Variables:**
- $\rho(t,x,y)$ : pedestrian density [ped/m²]
- $\psi(t,x,y)$ : transformed potential (via Cole-Hopf transformation: $\psi = e^{-\Phi/\delta}$)
- $u(t,x,y)$ : velocity field [m/s]

### 2.2 Governing Equations

**Transient Continuity Equation (with diffusion):**
$$
\frac{\partial\rho}{\partial t} - \varepsilon\Delta\rho + \nabla \cdot (\rho u) = 0 \quad \text{in } \Omega \times (0,T]
$$

**Helmholtz Equation (for path planning):**
$$
\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi = 0 \quad \text{in } \Omega
$$

**Velocity Field:**
$$
u = f(\rho) \frac{\nabla\psi}{\|\nabla\psi\|}
$$

where:
- $\varepsilon$ = diffusion coefficient (small, for numerical stability)
- $\delta$ = viscosity parameter (regularization parameter)
- $f(\rho)$ = fundamental diagram (walking speed as function of density)

### 2.3 Fundamental Diagram

We use Weidmann's fundamental diagram:
$$
f(\rho) = \begin{cases}
u_0 & \text{if } \rho = 0 \\
u_0\left(1 - e^{-\gamma(1/\rho - 1/\rho_c)}\right) & \text{if } 0 < \rho \leq \rho_c
\end{cases}
$$

**Parameters:**
- $u_0 = 1.36$ m/s (free-flow walking speed)
- $\rho_c = 8$ ped/m² (critical density)
- $\gamma = 1.913$ ped/m² (shape parameter)

### 2.4 Boundary Conditions

**On Walls ($\Gamma_{\text{walls}}$):**
- Density: $(-\varepsilon\nabla\rho + \rho u)\cdot n = 0$ (no flux through walls)
- Potential: $\nabla\psi\cdot n = 0$ (slip condition)

**On Exits ($\Gamma_{\text{exits}}$):**
- Density: $(-\varepsilon\nabla\rho)\cdot n = 0$ (free outflow)
- Potential: $\psi = 1$ (Dirichlet BC, minimal travel time at exit)

**On Entrances ($\Gamma_{\text{entries}}$):**
- Density: $-(-\varepsilon\nabla\rho + \rho u)\cdot n = g$ (prescribed influx)
- Potential: $(u_0\delta\nabla\psi)\cdot n + \psi = 0$ (Robin BC)

where:
- $n$ = outward normal vector on boundary
- $g$ = inflowing flux density [ped/(m·s)]

### 2.5 Initial Condition

**At $t = 0$:**
$$
\rho(0, x, y) = \rho_0(x, y) \quad \text{in } \Omega
$$

---

## 3. Weak Formulation Derivation

### 3.1 Weak Form of Helmholtz Equation

#### Step 1: Function Spaces

Define:
- Solution space: $\psi \in V_\psi = \{v \in H^1(\Omega) : v = 1 \text{ on } \Gamma_{\text{exits}}\}$
- Test space: $\varphi \in V_{0,\psi} = \{v \in H^1(\Omega) : v = 0 \text{ on } \Gamma_{\text{exits}}\}$

#### Step 2: Multiply by Test Function and Integrate

Starting from the strong form:
$$
\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi = 0
$$

Multiply by $\varphi \in V_{0,\psi}$ and integrate over $\Omega$:
$$
\int_\Omega \left(\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi\right) \varphi \, d\Omega = 0
$$

#### Step 3: Integration by Parts (Green's First Identity)

Apply Green's first identity to the Laplacian term:
$$
\int_\Omega \Delta\psi \, \varphi \, d\Omega = -\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS
$$

Substituting back:
$$
-\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega - \int_\Omega \frac{1}{\delta^2 f^2(\rho)} \psi\varphi \, d\Omega + \int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS = 0
$$

#### Step 4: Apply Boundary Conditions

Analyze the boundary integral $\int_{\partial\Omega} (\nabla\psi \cdot n) \varphi \, dS$:

**On $\Gamma_{\text{walls}}$:**
- Boundary condition: $\nabla\psi \cdot n = 0$
- Contribution: $\int_{\Gamma_{\text{walls}}} (\nabla\psi \cdot n) \varphi \, dS = 0$

**On $\Gamma_{\text{exits}}$:**
- Test function: $\varphi = 0$ (homogeneous Dirichlet)
- Contribution: $\int_{\Gamma_{\text{exits}}} (\nabla\psi \cdot n) \varphi \, dS = 0$

**On $\Gamma_{\text{entries}}$:**
- Boundary condition: $(u_0\delta\nabla\psi)\cdot n + \psi = 0 \Rightarrow \nabla\psi \cdot n = -\frac{\psi}{u_0\delta}$
- Contribution: $\int_{\Gamma_{\text{entries}}} (\nabla\psi \cdot n) \varphi \, dS = \int_{\Gamma_{\text{entries}}} \left(-\frac{\psi}{u_0\delta}\right) \varphi \, dS$

#### Step 5: Final Weak Form for Helmholtz Equation

**Find $\psi \in V_\psi$ such that for all $\varphi \in V_{0,\psi}$:**
$$
\boxed{
\int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho) \psi\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi\varphi \, dS = 0
}
$$

where $\kappa(\rho) = \frac{1}{\delta f(\rho)}$ is the wave number.

---

### 3.2 Weak Form of Transient Continuity Equation (Implicit Forward Euler)

#### Step 1: Time Discretization

We discretize the time interval $[0, T]$ into $N$ time steps with time step size $\Delta t = T/N$:
$$
t_n = n \Delta t, \quad n = 0, 1, 2, \ldots, N
$$

Let $\rho^n(x,y) = \rho(t_n, x, y)$ denote the density at time $t_n$.

**Implicit Forward Euler discretization** of the transient continuity equation:
$$
\frac{\rho^{n+1} - \rho^n}{\Delta t} - \varepsilon\Delta\rho^{n+1} + \nabla \cdot (\rho^{n+1} u^{n+1}) = 0
$$

Rearranging:
$$
\rho^{n+1} - \varepsilon\Delta t\Delta\rho^{n+1} + \Delta t\nabla \cdot (\rho^{n+1} u^{n+1}) = \rho^n
$$

#### Step 2: Function Spaces

Define:
- Solution space: $\rho^{n+1} \in V_\rho = H^1(\Omega)$ (no Dirichlet BCs)
- Test space: $w \in V_\rho = H^1(\Omega)$

#### Step 3: Multiply by Test Function and Integrate

Multiply the rearranged equation by $w \in V_\rho$ and integrate over $\Omega$:
$$
\int_\Omega \rho^{n+1} w \, d\Omega - \varepsilon\Delta t\int_\Omega \Delta\rho^{n+1} w \, d\Omega + \Delta t\int_\Omega \nabla \cdot (\rho^{n+1} u^{n+1}) w \, d\Omega = \int_\Omega \rho^n w \, d\Omega
$$

#### Step 4: Integration by Parts

Apply Green's first identity to the Laplacian term:
$$
-\varepsilon\Delta t\int_\Omega \Delta\rho^{n+1} w \, d\Omega = \varepsilon\Delta t\int_\Omega \nabla\rho^{n+1} \cdot \nabla w \, d\Omega - \varepsilon\Delta t\int_{\partial\Omega} (\nabla\rho^{n+1} \cdot n) w \, dS
$$

Apply divergence theorem to the convection term:
$$
\Delta t\int_\Omega \nabla \cdot (\rho^{n+1} u^{n+1}) w \, d\Omega = -\Delta t\int_\Omega (\rho^{n+1} u^{n+1}) \cdot \nabla w \, d\Omega + \Delta t\int_{\partial\Omega} (\rho^{n+1} u^{n+1} \cdot n) w \, dS
$$

Substituting back:
$$
\begin{align}
&\int_\Omega \rho^{n+1} w \, d\Omega + \varepsilon\Delta t\int_\Omega \nabla\rho^{n+1} \cdot \nabla w \, d\Omega - \Delta t\int_\Omega (\rho^{n+1} u^{n+1}) \cdot \nabla w \, d\Omega \\
&\quad - \varepsilon\Delta t\int_{\partial\Omega} (\nabla\rho^{n+1} \cdot n) w \, dS + \Delta t\int_{\partial\Omega} (\rho^{n+1} u^{n+1} \cdot n) w \, dS = \int_\Omega \rho^n w \, d\Omega
\end{align}
$$

#### Step 5: Apply Boundary Conditions

Analyze the boundary integrals:

**On $\Gamma_{\text{walls}}$:**
- Boundary condition: $(-\varepsilon\nabla\rho + \rho u) \cdot n = 0$
- This gives: $-\varepsilon(\nabla\rho^{n+1} \cdot n) + (\rho^{n+1} u^{n+1} \cdot n) = 0$
- Both boundary integrals cancel

**On $\Gamma_{\text{exits}}$:**
- Boundary condition: $(-\varepsilon\nabla\rho) \cdot n = 0$, so $\nabla\rho^{n+1} \cdot n = 0$
- Contribution: $\Delta t\int_{\Gamma_{\text{exits}}} (\rho^{n+1} u^{n+1} \cdot n) w \, dS$

**On $\Gamma_{\text{entries}}$:**
- Boundary condition: $-(-\varepsilon\nabla\rho + \rho u) \cdot n = g$
- This gives: $\varepsilon(\nabla\rho^{n+1} \cdot n) - (\rho^{n+1} u^{n+1} \cdot n) = g$
- Contribution: $-\Delta t\int_{\Gamma_{\text{entries}}} g w \, dS$

#### Step 6: Final Weak Form for Transient Continuity Equation

**Find $\rho^{n+1} \in V_\rho$ such that for all $w \in V_\rho$:**
$$
\boxed{
\begin{align}
&\int_\Omega \rho^{n+1} w \, d\Omega + \varepsilon\Delta t\int_\Omega \nabla\rho^{n+1} \cdot \nabla w \, d\Omega - \Delta t\int_\Omega (\rho^{n+1} u^{n+1}) \cdot \nabla w \, d\Omega \\
&\quad + \Delta t\int_{\Gamma_{\text{exits}}} (\rho^{n+1} u^{n+1} \cdot n) w \, dS = \int_\Omega \rho^n w \, d\Omega + \Delta t\int_{\Gamma_{\text{entries}}} g w \, dS
\end{align}
}
$$

---

## 4. Coupled System Solution Strategy

The system is coupled and nonlinear at each time step. We solve it using a fixed-point iteration (Picard iteration) within each time step:

### Algorithm

**Initialize:** Start with initial density $\rho^0(x,y) = \rho_0(x,y)$

**Time stepping:** For $n = 0, 1, 2, \ldots, N-1$:

1. Set $\rho^{n+1,(0)} = \rho^n$ (initial guess for new time step)

2. **Picard Iteration:** For $k = 0, 1, 2, \ldots$ until convergence:

   a. **Solve Helmholtz equation** for $\psi^{n+1,(k+1)}$ using $\rho^{n+1,(k)}$:
   $$
   \int_\Omega \nabla\psi^{n+1,(k+1)} \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho^{n+1,(k)}) \psi^{n+1,(k+1)}\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi^{n+1,(k+1)}\varphi \, dS = 0
   $$

   b. **Compute velocity field** $u^{n+1,(k+1)}$:
   $$
   u^{n+1,(k+1)} = f(\rho^{n+1,(k)}) \frac{\nabla\psi^{n+1,(k+1)}}{\|\nabla\psi^{n+1,(k+1)}\| + \eta}
   $$
   where $\eta > 0$ is a small regularization parameter to avoid division by zero.

   c. **Solve continuity equation** for $\rho^{n+1,(k+1)}$ using $u^{n+1,(k+1)}$:
   $$
   \begin{align}
   &\int_\Omega \rho^{n+1,(k+1)} w \, d\Omega + \varepsilon\Delta t\int_\Omega \nabla\rho^{n+1,(k+1)} \cdot \nabla w \, d\Omega - \Delta t\int_\Omega (\rho^{n+1,(k+1)} u^{n+1,(k+1)}) \cdot \nabla w \, d\Omega \\
   &\quad + \Delta t\int_{\Gamma_{\text{exits}}} (\rho^{n+1,(k+1)} u^{n+1,(k+1)} \cdot n) w \, dS = \int_\Omega \rho^n w \, d\Omega + \Delta t\int_{\Gamma_{\text{entries}}} g w \, dS
   \end{align}
   $$

   d. **Check convergence:** If $\|\rho^{n+1,(k+1)} - \rho^{n+1,(k)}\| < \text{tol}$, set $\rho^{n+1} = \rho^{n+1,(k+1)}$ and proceed to next time step.

---

## 5. Summary of Weak Forms for NGSolve Implementation

### Helmholtz Equation (for $\psi$)
**Bilinear form:**
$$
a_\psi(\psi, \varphi) = \int_\Omega \nabla\psi \cdot \nabla\varphi \, d\Omega + \int_\Omega \kappa^2(\rho) \psi\varphi \, d\Omega + \int_{\Gamma_{\text{entries}}} \frac{1}{u_0\delta} \psi\varphi \, dS
$$

**Linear form:**
$$
l_\psi(\varphi) = 0
$$

**Essential BC:** $\psi = 1$ on $\Gamma_{\text{exits}}$

### Transient Continuity Equation (for $\rho^{n+1}$)
**Bilinear form:**
$$
a_\rho(\rho^{n+1}, w) = \int_\Omega \rho^{n+1} w \, d\Omega + \varepsilon\Delta t\int_\Omega \nabla\rho^{n+1} \cdot \nabla w \, d\Omega - \Delta t\int_\Omega (\rho^{n+1} u^{n+1}) \cdot \nabla w \, d\Omega + \Delta t\int_{\Gamma_{\text{exits}}} (\rho^{n+1} u^{n+1} \cdot n) w \, dS
$$

**Linear form:**
$$
l_\rho(w) = \int_\Omega \rho^n w \, d\Omega + \Delta t\int_{\Gamma_{\text{entries}}} g w \, dS
$$

**No essential BCs** (all boundary conditions are natural/Neumann-type)

---

## 6. Next Steps

In the following cells, we will:
1. Import NGSolve libraries
2. Define geometry and mesh
3. Set up finite element spaces
4. Implement the weak forms with time stepping
5. Implement the iterative solver with time loop
6. Visualize results over time
7. Verify conservation and physical correctness

In [11]:
# ========================================
# Module Import
# ========================================

from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np

In [12]:
# ========================================
# Geometry and BC naming
# ========================================

mesh_maxh = 0.05
Hcol = 1.0  # Height [m]
Hwid = 1.0  # Width [m]

# Top rectangle
rect = Rectangle(Hwid, Hcol).Face()
rect.edges.Min(X).name = "left"
rect.edges.Max(X).name = "right"
rect.edges.Max(Y).name = "entry"
rect.edges.Min(Y).name = "exit"

geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")

Mesh: 514 vertices, 946 elements


## Model Parameters and Constants

We now define all physical parameters and numerical constants for the pedestrian flow model.

In [ ]:
# ========================================
# Physical Parameters (from Weidmann)
# ========================================
u0 = 1.36          # Free-flow walking speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

# ========================================
# Regularization Parameters
# ========================================
delta = 0.1        # Viscosity parameter [m]
epsilon = 0.075     # Diffusion coefficient [m²]
eta = 1e-8         # Gradient regularization (avoid division by zero)

# ========================================
# Boundary Conditions
# ========================================
g_inflow = 1.0     # Influx at entrance [ped/(m·s)]

# ========================================
# Time Discretization Parameters
# ========================================
t_end = 5.0        # End time [s]
dt = 0.01          # Initial time step [s]
dt_min = 0.001     # Minimum allowed time step [s]
dt_max = 0.1       # Maximum allowed time step [s]

# ========================================
# Numerical Parameters
# ========================================
p_order = 3        # Polynomial order of FEM spaces
max_iter = 100     # Maximum number of Picard iterations
tol = 1e-6         # Convergence tolerance for ||rho^(k+1) - rho^(k)||

print("Model Parameters:")
print(f"  u0 (free-flow speed)     = {u0} m/s")
print(f"  rho_c (critical density) = {rho_c} ped/m²")
print(f"  gamma (Weidmann param)   = {gamma_w} ped/m²")
print(f"\nRegularization:")
print(f"  delta (viscosity)        = {delta} m")
print(f"  epsilon (diffusion)      = {epsilon} m²")
print(f"  eta (grad regularization)= {eta}")
print(f"\nBoundary Conditions:")
print(f"  g_inflow (entrance flux) = {g_inflow} ped/(m·s)")
print(f"\nTime Discretization:")
print(f"  t_end                    = {t_end} s")
print(f"  dt (initial)             = {dt} s")
print(f"  dt range                 = [{dt_min}, {dt_max}] s")
print(f"\nNumerical Settings:")
print(f"  max_iter                 = {max_iter}")
print(f"  tolerance                = {tol}")

In [14]:
# ============================================================================
# PARAMETER ANALYSIS - Verify numerical stability before running solver
# ============================================================================
from src import analyze_parameters

print("Running parameter analysis...\n")

results = analyze_parameters(
    u0=u0,
    rho_c=rho_c,
    gamma_w=gamma_w,
    delta=delta,
    epsilon=epsilon,
    mesh=mesh,
    mesh_maxh=mesh_maxh,
    p_order=p_order,
    Hwid=Hwid,
    Hcol=Hcol,
    omega=None  # No underrelaxation in standard Picard
)

# Check if parameters are acceptable
if results['stability_continuity'] in ['EXCELLENT', 'GOOD']:
    print("\n" + "="*60)
    print("✓✓ Parameters are GOOD! Ready to proceed with simulation.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠⚠ WARNING: Parameters may cause stability issues!")
    print("    Consider adjusting epsilon, h, or p before continuing.")
    print("="*60)

Running parameter analysis...

PARAMETER ANALYSIS AND NUMERICAL RECOMMENDATIONS

1. PHYSICAL PARAMETERS

Fundamental Diagram:
  u₀ (free-flow speed)      = 1.360 m/s
  ρc (critical density)     = 8.0 ped/m²
  γ  (Weidmann parameter)   = 1.913 ped/m²

Regularization:
  δ (Helmholtz viscosity)   = 0.100 m
  ε (continuity diffusion)  = 0.075 m²

Speed at different densities:
  f(0.5) = 1.322 m/s
  f(1.0) = 1.105 m/s
  f(2.0) = 0.696 m/s
  f(4.0) = 0.289 m/s
  f(6.0) = 0.104 m/s

2. CHARACTERISTIC LENGTH SCALES

Domain:
  Width × Height = 1.00 × 1.00 m
  L (characteristic) = 1.00 m

Diffusion length (continuity):
  lε = √(ε·L/u₀) = 0.2348 m = 23.48 cm
  Physical meaning: density spreads ~23.5 cm

Screening length (Helmholtz):
  lδ = δ·u₀ = 0.1360 m = 13.60 cm
  Physical meaning: path smoothing ~13.6 cm

Helmholtz wavelength λ = 2π·δ·f(ρ):
  λ(ρ=0.5) = 0.8309 m = 83.09 cm
  λ(ρ=1.0) = 0.6943 m = 69.43 cm
  λ(ρ=2.0) = 0.4375 m = 43.75 cm
  λ(ρ=4.0) = 0.1817 m = 18.17 cm
  λ(ρ=6.0) = 0.0655 m

In [15]:
# ========================================
# Weidmann Fundamental Diagram
# ========================================

def weidmann_speed(rho_val):
    """
    Weidmann fundamental diagram: f(rho)
    
    Returns walking speed as a function of density.
    
    f(rho) = u0 * (1 - exp(-gamma * (1/rho - 1/rho_c)))  for 0 < rho <= rho_c
    f(0)   = u0                                           for rho = 0
    
    Parameters:
    -----------
    rho_val : float or CoefficientFunction
        Pedestrian density [ped/m²]
    
    Returns:
    --------
    speed : float or CoefficientFunction
        Walking speed [m/s]
    """
    # Add small regularization to avoid division by zero
    rho_reg = IfPos(rho_val - 1e-10, rho_val, 1e-10)
    
    # Weidmann formula
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    # Ensure speed doesn't exceed u0 (free-flow speed)
    speed = IfPos(u0 - speed, speed, u0)
    
    # Ensure speed is non-negative
    speed = IfPos(speed, speed, 0.0)
    
    return speed


# Test function for pure Python evaluation (for testing)
def weidmann_speed_python(rho_val):
    """Python version for numerical testing"""
    rho_reg = max(rho_val, 1e-10)
    speed = u0 * (1 - np.exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    speed = min(max(speed, 0.0), u0)
    return speed


# Test the function with some sample values
print("Testing Weidmann fundamental diagram:")
print(f"  f(0.1) = {weidmann_speed_python(0.1):.4f} m/s")
print(f"  f(1.0) = {weidmann_speed_python(1.0):.4f} m/s")
print(f"  f(2.0) = {weidmann_speed_python(2.0):.4f} m/s")
print(f"  f(4.0) = {weidmann_speed_python(4.0):.4f} m/s")
print(f"  f(8.0) = {weidmann_speed_python(rho_c):.4f} m/s  (at critical density)")
print(f"  f(0.0) ≈ {weidmann_speed_python(1e-12):.4f} m/s  (near zero)")

Testing Weidmann fundamental diagram:
  f(0.1) = 1.3600 m/s
  f(1.0) = 1.1050 m/s
  f(2.0) = 0.6963 m/s
  f(4.0) = 0.2892 m/s
  f(8.0) = 0.0000 m/s  (at critical density)
  f(0.0) ≈ 1.3600 m/s  (near zero)


In [ ]:
# ========================================
# Finite Element Spaces
# ========================================

# H1 space for density (rho) - no Dirichlet BCs
fes_rho = H1(mesh, order=p_order)

# H1 space for potential (psi) - Dirichlet BC on exit
fes_psi = H1(mesh, order=p_order, dirichlet="right")

# Vector H1 space for velocity field (u)
fes_u = H1(mesh, order=p_order, dim=2)

print("Finite Element Spaces:")
print(f"  Density (rho):    {fes_rho.ndof} DOFs")
print(f"  Potential (psi):  {fes_psi.ndof} DOFs ({fes_psi.ndof - fes_rho.ndof} fixed by Dirichlet BC)")
print(f"  Velocity (u):     {fes_u.ndof} DOFs")

# ========================================
# Grid Functions (Solution Variables)
# ========================================

# Current solution at time t^{n+1}
gf_rho = GridFunction(fes_rho, name="density")
gf_psi = GridFunction(fes_psi, name="potential")
gf_u = GridFunction(fes_u, name="velocity")

# Previous time step solution at time t^n
gf_rho_prev = GridFunction(fes_rho, name="density_prev")

# Picard iteration storage (old iterate within time step)
gf_rho_old = GridFunction(fes_rho, name="density_old")

# ========================================
# Initialize with simple initial condition
# ========================================

# Simple initial condition: low density everywhere
rho_init = 0.1

gf_rho.Set(rho_init)
gf_rho_prev.Set(rho_init)
gf_rho_old.Set(rho_init)

# Initialize potential with linear interpolation from entry (0) to exit (1)
gf_psi.Set(y / Hcol)  

# Set Dirichlet BC for psi: psi = 1 at exit
gf_psi.Set(1.0, definedon=mesh.Boundaries("right"))

# Velocity will be computed from psi
gf_u.Set((0, 0))

print("\nGrid Functions initialized:")
print(f"  Initial density: ρ₀ = {rho_init} ped/m² (uniform)")
print(f"  rho: min = {min(gf_rho.vec):.4f}, max = {max(gf_rho.vec):.4f}")
print(f"  psi: min = {min(gf_psi.vec):.4f}, max = {max(gf_psi.vec):.4f}")

In [17]:
# ========================================
# Helmholtz Equation Weak Form (for ψ)
# ========================================
# Weak form:
# ∫_Ω ∇ψ·∇φ dΩ + ∫_Ω κ²(ρ) ψφ dΩ + ∫_Γ_entry (1/(u0*δ)) ψφ dS = 0
# where κ²(ρ) = 1/(δ²f²(ρ))

print("Setting up Helmholtz equation weak form...")

# Trial and test functions
psi = fes_psi.TrialFunction()
phi = fes_psi.TestFunction()

# Coefficient function for κ²(ρ) based on current density
# κ²(ρ) = 1/(δ² * f²(ρ))
f_rho = weidmann_speed(gf_rho)  # Speed as function of current density
kappa_sq = 1.0 / (delta**2 * f_rho**2)

# Bilinear form for Helmholtz equation
a_psi = BilinearForm(fes_psi, symmetric=True)

# Volume terms: ∫_Ω ∇ψ·∇φ dΩ + ∫_Ω κ²(ρ) ψφ dΩ
a_psi += grad(psi) * grad(phi) * dx
a_psi += kappa_sq * psi * phi * dx

# Robin BC on entry: ∫_Γ_entry (1/(u0*δ)) ψφ dS
a_psi += (1.0 / (u0 * delta)) * psi * phi * ds("entry")

# Linear form (RHS) - is zero for Helmholtz
L_psi = LinearForm(fes_psi)
# (no terms, it's zero)

print("  Helmholtz weak form created")
print(f"  - Gradient term: ∫ ∇ψ·∇φ dx")
print(f"  - Helmholtz term: ∫ κ²(ρ) ψφ dx  with κ² = 1/(δ²f²(ρ))")
print(f"  - Robin BC: ∫ (1/(u0*δ)) ψφ ds('entry')")
print(f"  - RHS: 0")

# ========================================
# Continuity Equation Weak Form (for ρ)
# ========================================
# Weak form:
# ∫_Ω ε∇ρ·∇w dΩ - ∫_Ω (ρu)·∇w dΩ + ∫_Γ_exit (ρu·n)w dS = ∫_Γ_entry g w dS

print("\nSetting up Continuity equation weak form...")

# Trial and test functions
rho = fes_rho.TrialFunction()
w = fes_rho.TestFunction()

# Bilinear form for Continuity equation
a_rho = BilinearForm(fes_rho, symmetric=False)  # Not symmetric due to convection

# Diffusion term: ∫_Ω ε∇ρ·∇w dΩ
a_rho += epsilon * grad(rho) * grad(w) * dx

# Convection term: - ∫_Ω (ρu)·∇w dΩ
# Note: gf_u will be updated in the iteration loop
a_rho += -rho * (gf_u * grad(w)) * dx

# Exit boundary term: ∫_Γ_exit (ρu·n)w dS
# Note: n is the outward normal, automatically handled by ds
n = specialcf.normal(2)  # 2D normal vector
a_rho += rho * (gf_u * n) * w * ds("right")

# Linear form (RHS) - inflow at entry
L_rho = LinearForm(fes_rho)
L_rho += g_inflow * w * ds("entry")

print("  Continuity weak form created")
print(f"  - Diffusion term: ∫ ε∇ρ·∇w dx")
print(f"  - Convection term: -∫ (ρu)·∇w dx")
print(f"  - Exit BC: ∫ (ρu·n)w ds('exit')")
print(f"  - RHS: ∫ g w ds('entry') with g = {g_inflow}")

print("\n✓ Weak forms setup complete")

Setting up Helmholtz equation weak form...
  Helmholtz weak form created
  - Gradient term: ∫ ∇ψ·∇φ dx
  - Helmholtz term: ∫ κ²(ρ) ψφ dx  with κ² = 1/(δ²f²(ρ))
  - Robin BC: ∫ (1/(u0*δ)) ψφ ds('entry')
  - RHS: 0

Setting up Continuity equation weak form...
  Continuity weak form created
  - Diffusion term: ∫ ε∇ρ·∇w dx
  - Convection term: -∫ (ρu)·∇w dx
  - Exit BC: ∫ (ρu·n)w ds('exit')
  - RHS: ∫ g w ds('entry') with g = 1.0

✓ Weak forms setup complete


In [18]:
# ========================================
# Velocity Field Computation
# ========================================
# Computes: u = f(ρ) * ∇ψ / (||∇ψ|| + η)
# where f(ρ) is the Weidmann speed and η is a regularization parameter

def compute_velocity():
    """
    Compute velocity field from current potential and density.
    
    Updates gf_u based on:
        u = f(rho) * grad(psi) / (||grad(psi)|| + eta)
    
    This is the coupling between Helmholtz and Continuity equations.
    """
    # Compute gradient of potential
    grad_psi = grad(gf_psi)
    
    # Compute norm of gradient with regularization
    grad_psi_norm = sqrt(grad_psi[0]**2 + grad_psi[1]**2 + eta)
    
    # Compute speed from current density
    speed = weidmann_speed(gf_rho)
    
    # Compute velocity: u = f(rho) * grad(psi) / ||grad(psi)||
    velocity_cf = speed * grad_psi / grad_psi_norm
    
    # Set the velocity grid function
    gf_u.Set(velocity_cf)
    
    return gf_u


print("Velocity field computation function defined:")
print("  u = f(ρ) * ∇ψ / (||∇ψ|| + η)")
print(f"  - f(ρ): Weidmann fundamental diagram")
print(f"  - η = {eta} (gradient regularization)")
print("\n✓ Velocity computation ready")

Velocity field computation function defined:
  u = f(ρ) * ∇ψ / (||∇ψ|| + η)
  - f(ρ): Weidmann fundamental diagram
  - η = 1e-08 (gradient regularization)

✓ Velocity computation ready


In [ ]:
# ========================================
# Transient Picard Solver with Time Loop
# ========================================

def solve_transient_picard():
    """
    Solve the coupled transient pedestrian flow problem using implicit Euler + Picard iteration.
    
    Algorithm:
    1. Initialize with gf_rho at t=0
    2. Time loop: for each time step t^n → t^{n+1}
       a. Set gf_rho_prev = gf_rho (store previous time step)
       b. Picard iteration at current time step:
          i.   Solve Helmholtz for psi
          ii.  Compute velocity u
          iii. Solve transient continuity for rho (with mass matrix)
          iv.  Check Picard convergence
       c. Advance time
    
    Returns:
    --------
    t_history : list
        Time values
    rho_max_history : list
        Maximum density at each time step
    """
    
    print("\n" + "="*70)
    print("TRANSIENT PICARD SOLVER - Implicit Euler Time Integration")
    print("="*70)
    print(f"Time interval: [0, {t_end}] s")
    print(f"Initial dt: {dt} s")
    print(f"DOFs: {fes_rho.ndof}")
    print("="*70 + "\n")
    
    # Initialize time and step counter
    t = 0.0
    istep = 0
    dt_current = dt
    
    # Storage for history
    t_history = [t]
    rho_max_history = [max(gf_rho.vec)]
    
    # Trial and test functions
    psi_trial = fes_psi.TrialFunction()
    phi_test = fes_psi.TestFunction()
    rho_trial = fes_rho.TrialFunction()
    w_test = fes_rho.TestFunction()
    
    # Normal vector for boundary integrals
    n = specialcf.normal(2)
    
    # ========================================
    # MAIN TIME LOOP
    # ========================================
    while t < t_end - 1e-12:
        istep += 1
        
        # Store previous time step solution
        gf_rho_prev.vec.data = gf_rho.vec
        
        # Initialize Picard iteration for this time step
        # (use previous time step as initial guess)
        picard_converged = False
        
        print(f"[Step {istep:3d}] t = {t:.4f} s → {t+dt_current:.4f} s (dt = {dt_current:.4e} s)")
        
        # ====================================
        # PICARD ITERATION LOOP
        # ====================================
        for picard_iter in range(max_iter):
            
            # Store old Picard iterate
            gf_rho_old.vec.data = gf_rho.vec
            
            # ----------------------------------------
            # (a) Solve Helmholtz equation for ψ
            # ----------------------------------------
            f_rho = weidmann_speed(gf_rho)
            kappa_sq = 1.0 / (delta**2 * f_rho**2)
            
            a_psi = BilinearForm(fes_psi, symmetric=True)
            a_psi += grad(psi_trial) * grad(phi_test) * dx
            a_psi += kappa_sq * psi_trial * phi_test * dx
            a_psi += (1.0 / (u0 * delta)) * psi_trial * phi_test * ds("entry")
            
            L_psi = LinearForm(fes_psi)
            
            a_psi.Assemble()
            L_psi.Assemble()
            
            # Apply Dirichlet BC: psi = 1 on exit
            gf_psi.Set(1.0, definedon=mesh.Boundaries("right"))
            
            # Solve linear system
            gf_psi.vec.data += a_psi.mat.Inverse(fes_psi.FreeDofs()) * (L_psi.vec - a_psi.mat * gf_psi.vec)
            
            # ----------------------------------------
            # (b) Compute velocity field u
            # ----------------------------------------
            compute_velocity()
            
            # ----------------------------------------
            # (c) Solve TRANSIENT continuity equation for ρ^{n+1}
            #     With MASS MATRIX term: (ρ^{n+1} - ρ^n)/Δt
            # ----------------------------------------
            a_rho = BilinearForm(fes_rho, symmetric=False)
            
            # MASS MATRIX: ∫ ρ^{n+1} w dx
            a_rho += rho_trial * w_test * dx
            
            # Diffusion: Δt * ∫ ε ∇ρ^{n+1} · ∇w dx
            a_rho += dt_current * epsilon * grad(rho_trial) * grad(w_test) * dx
            
            # Convection: -Δt * ∫ (ρ^{n+1} u^{n+1}) · ∇w dx
            a_rho += -dt_current * rho_trial * (gf_u * grad(w_test)) * dx
            
            # Exit BC: Δt * ∫ (ρ^{n+1} u^{n+1} · n) w dS
            a_rho += dt_current * rho_trial * (gf_u * n) * w_test * ds("right")
            
            # Linear form (RHS)
            L_rho = LinearForm(fes_rho)
            
            # Previous time step: ∫ ρ^n w dx
            L_rho += gf_rho_prev * w_test * dx
            
            # Inflow BC: Δt * ∫ g w dS
            L_rho += dt_current * g_inflow * w_test * ds("entry")
            
            # Assemble and solve
            with TaskManager():
                a_rho.Assemble()
                L_rho.Assemble()
                
                # Solve (no Dirichlet BCs for rho)
                gf_rho.vec.data = a_rho.mat.Inverse(fes_rho.FreeDofs()) * L_rho.vec
            
            # ----------------------------------------
            # (d) Check Picard convergence
            # ----------------------------------------
            residual_vec = gf_rho.vec - gf_rho_old.vec
            residual = sqrt(InnerProduct(residual_vec, residual_vec))
            
            if picard_iter % 5 == 0 or residual < tol:
                rho_min = min(gf_rho.vec)
                rho_max = max(gf_rho.vec)
                print(f"    Picard {picard_iter:2d}: residual = {residual:.3e}, "
                      f"ρ ∈ [{rho_min:.3f}, {rho_max:.3f}]")
            
            if residual < tol:
                print(f"    → Converged in {picard_iter+1} Picard iterations")
                picard_converged = True
                break
        
        if not picard_converged:
            print(f"    ⚠ Picard did not converge in {max_iter} iterations!")
            print(f"       Trying to reduce time step...")
            dt_current = max(0.5 * dt_current, dt_min)
            if dt_current <= dt_min + 1e-12:
                raise RuntimeError("Hit minimum time step, cannot continue")
            # Rollback to previous time step
            gf_rho.vec.data = gf_rho_prev.vec
            continue
        
        # ====================================
        # Accept time step and advance
        # ====================================
        t += dt_current
        t_history.append(t)
        rho_max_history.append(max(gf_rho.vec))
        
        # Simple adaptive time stepping (optional)
        # If converged quickly, try to increase dt
        if picard_iter < 5:
            dt_current = min(1.2 * dt_current, dt_max)
        
        print()
    
    print("="*70)
    print(f"✓ Simulation complete: t = {t:.4f} s ({istep} time steps)")
    print("="*70)
    
    return t_history, rho_max_history


# Run the transient solver
t_history, rho_max_history = solve_transient_picard()

In [ ]:
# ========================================
# Visualize Final Solution and Time History
# ========================================

print("=" * 60)
print("Solution Visualization")
print("=" * 60)

# Compute domain area
domain_area = Integrate(1.0, mesh)

# Create order-1 space for min/max evaluation
fes_p1 = H1(mesh, order=1)

# Print final solution statistics
print("\nFinal Solution Statistics:")
print(f"  Time: t = {t_history[-1]:.4f} s")
print(f"  Density (ρ):")
rho_integral = Integrate(gf_rho * dx, mesh)
rho_mean = rho_integral / domain_area

# Interpolate to P1 for min/max
gf_rho_p1 = GridFunction(fes_p1)
gf_rho_p1.Set(gf_rho)

print(f"    integral = {rho_integral:.6f} ped")
print(f"    mean = {rho_mean:.6f} ped/m²")
print(f"    min ≈ {min(gf_rho_p1.vec):.6f} ped/m² (at vertices)")
print(f"    max ≈ {max(gf_rho_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Domain area: {domain_area:.6f} m²")

print("=" * 60)

# Draw the final fields
print("\nGenerating visualizations...")

# 1. Final density field
Draw(gf_rho, mesh, "density_final")
print("  ✓ Final density field (ρ)")

# 2. Final velocity vector field
Draw(gf_u, mesh, "velocity_final", vectors={"grid_size": 30})
print("  ✓ Final velocity vector field (u)")

# 3. Plot maximum density over time
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(t_history, rho_max_history, 'b-', linewidth=2)
plt.xlabel('Time [s]', fontsize=12)
plt.ylabel('Maximum Density [ped/m²]', fontsize=12)
plt.title('Evolution of Maximum Density', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print("  ✓ Time history plot")

print("\n✓ Visualization complete!")